# 1-create-la-flatfile

This notebook goes through the CIN Census files stored in cin_received (filepath defined in 00-config) and conducts the following steps:
- Degrades date of birth into year of birth and school year
- Cleans the CIN Census files (light touch - checks for empty tags, date in improper format, etc)
- Creates a flat CSV file containing all CIN events for the LA (one row per date event - referral, CIN start, CIN end, etc. - with demographic info and a few relevant info to the event) - this means that there are a LOT of columns.

In [11]:
import glob
import os
import yaml
from wrangling.cincensus.main import main

%run "00-config.ipynb"
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Config

In [12]:
localauthority = 'Wandsworth' # Write name of LA - this must match the folder name

#### Define input and output folder

In [13]:
input_folder = os.path.join(cin_folder, localauthority)
output_folder = flatfile_folder

print("Input folder: {} \nOutput folder: {}".format(input_folder, output_folder))

Input folder: C:\Users\CGross\Desktop\cin-census-testing\cincensus\Wandsworth 
Output folder: C:\Users\CGross\Desktop\cin-census-testing\flatfiles


In [14]:
# Config file
with open('wrangling/config/cin_datamap.yaml') as FILE:
    config = yaml.load(FILE, Loader=yaml.FullLoader)

### 1. Run programme (takes a few minutes depending on quantity of data)

In [25]:
main(input_folder, output_folder, config)

Found 1 CIN files in folder C:\Users\CGross\Desktop\cin-census-testing\cincensus\Wandsworth
File 1 out of 1
--- Degrade file 1
5338 PersonBirthDate events were found, of which 5338 were degraded to year of birth and school year
--- Clean file 1
--- Create CIN flatfile
Extracting data from file 1 out of 1 from CIN Census
Done!
